# Data Collection and cleansing

In [8]:
#get the data

from urllib.request import urlopen
import urllib.parse as up
import urllib.request as ur

#download all the csv.gz files on airbnb website only for Unted States and 2016. to filter the links on the airbnb webpage
# [http://insideairbnb.com/get-the-data.html] I used urllib library and to filter csv files to get only files for United States
#in 2016 I splits the url and download only these files.
url = 'http://insideairbnb.com/get-the-data.html'
from bs4 import BeautifulSoup as bs
K=0
html = urlopen(url).read()
soup = bs(html,"lxml")
tags = soup('a')
for tag in tags:
    href =(tag.get('href', None))

    try:

        if href.endswith("csv.gz") and href.find("united-states")!=-1 and href.split("/")[-3].split("-")[0]=="2016":

            ur.urlretrieve(href,href.split("/")[-4]+"_"+href.split("/")[-3]+"_"+href.split("/")[-1])
            print(href.split("/")[-4]+"_"+href.split("/")[-3]+"_"+href.split("/")[-1])

    except:

        continue



asheville_2016-04-18_listings.csv.gz
asheville_2016-04-18_calendar.csv.gz
asheville_2016-04-18_reviews.csv.gz
boston_2016-09-07_listings.csv.gz
boston_2016-09-07_calendar.csv.gz
boston_2016-09-07_reviews.csv.gz
denver_2016-05-16_listings.csv.gz
denver_2016-05-16_calendar.csv.gz
denver_2016-05-16_reviews.csv.gz
los-angeles_2016-08-03_listings.csv.gz
los-angeles_2016-08-03_calendar.csv.gz
los-angeles_2016-08-03_reviews.csv.gz
los-angeles_2016-07-03_listings.csv.gz
los-angeles_2016-07-03_calendar.csv.gz
los-angeles_2016-07-03_reviews.csv.gz
los-angeles_2016-05-02_listings.csv.gz
los-angeles_2016-05-02_calendar.csv.gz
los-angeles_2016-05-02_reviews.csv.gz
los-angeles_2016-01-02_listings.csv.gz
los-angeles_2016-01-02_calendar.csv.gz
los-angeles_2016-01-02_reviews.csv.gz
nashville_2016-09-06_listings.csv.gz
nashville_2016-09-06_calendar.csv.gz
nashville_2016-09-06_reviews.csv.gz
new-orleans_2016-10-03_listings.csv.gz
new-orleans_2016-10-03_calendar.csv.gz
new-orleans_2016-10-03_reviews.csv.g

In [ ]:
#concat all csv files across all cities for different file types 
import pandas as pd
import glob
import os

#loadfiles() function gets the file type and loop through all the folders and append those filetypes to filetype+"_all" file 
#in finaldata folder

def loadfiles(filetype):
    df = pd.DataFrame([])
    k=0
    for filename in glob.iglob("*_"+filetype+".csv.gz"):
        df2 = pd.DataFrame([])
        df2= pd.read_csv(filename)
        #add city and file_date derived from filename to the dataframe
        df2["city"],df2["file_date"]=filename.split("_")[0],filename.split("_")[1]
        if not os.path.isfile("finaldata\\"+filetype+"_all.csv"):
            df2.to_csv("finaldata\\"+filetype+"_all.csv",mode='a')
        else:
            df2.to_csv("finaldata\\"+filetype+"_all.csv",mode='a', header=False)
        k=k+1
        print(k,filename)
    

loadfiles("reviews")
loadfiles("listings")
loadfiles("calendar")

In [84]:
#preparing data
import numpy as np

listing=pd.read_csv("finaldata/listings_all.csv", encoding = "ISO-8859-1",low_memory=False)


#converting price datatype to int. Some of the prices is saved with dollar signs and commas
if listing['price'].dtype!='int64' and listing['price'][0]=="$":
    listing['price']=listing['price'].map(lambda p: int(p[1:-3].replace(",", "")))

In [48]:
#In this step we want to know about presence or absence of amenities a home offers. To do that we create a dictionary of 
#all amenities offered by all homes then compare each home with this list and give True and False values

In [85]:
#clean the amenities column. Amenities column is saved as a set so we need to get the members of this set in a list
listing['amenities'] = listing['amenities'].apply( 
    lambda amns: "|".join([amn.replace("}", "").replace("{", "").replace('"', "") for amn in amns.split("|")]) if isinstance(amns, str) else amns
)

In [61]:
import numpy as np
np.concatenate(listing['amenities'].apply(lambda amns: amns.split("|")).values)

array([ 'TV,Wireless Internet,Kitchen,Free Parking on Premises,Pets live on this property,Dog(s),Heating,Family/Kid Friendly,Washer,Dryer,Smoke Detector,Fire Extinguisher,Essentials,Shampoo,Laptop Friendly Workspace',
       'TV,Internet,Wireless Internet,Air Conditioning,Kitchen,Pets Allowed,Pets live on this property,Dog(s),Heating,Family/Kid Friendly,Washer,Dryer,Smoke Detector,Carbon Monoxide Detector,Fire Extinguisher,Essentials,Shampoo,Lock on Bedroom Door,Hangers,Hair Dryer,Iron',
       'TV,Cable TV,Wireless Internet,Air Conditioning,Kitchen,Free Parking on Premises,Heating,Washer,Dryer,Smoke Detector,Carbon Monoxide Detector,First Aid Kit,Safety Card,Essentials,Shampoo,Lock on Bedroom Door,translation missing: en.hosting_amenity_49,translation missing: en.hosting_amenity_50',
       ..., 'TV,Smoke Detector,Fire Extinguisher,Essentials',
       'Wireless Internet,Air Conditioning,Kitchen,Free Parking on Premises,Heating,Washer,Dryer,Smoke Detector,Carbon Monoxide Detector,Essen

In [86]:
#get a list of unique amenities
amenities = np.unique(np.concatenate(listing['amenities'].apply(lambda amns: amns.split(",")).values))

#check if a listing has a specific amenities from the amenities dictionary.
amenities_matrix = np.array([listing['amenities'].apply(lambda amns: amn in amns).values for amn in amenities])
amenities_matrix

array([[ True,  True,  True, ...,  True,  True,  True],
       [False, False, False, ..., False,  True, False],
       [False, False, False, ..., False, False, False],
       ..., 
       [ True,  True,  True, ..., False,  True,  True],
       [False, False,  True, ..., False, False,  True],
       [False, False,  True, ..., False, False,  True]], dtype=bool)

In [87]:
#pick some of the columns from df dataframe and creat a new dataframe
features=listing[['host_listings_count', 'host_total_listings_count', 'accommodates', 
                     'bathrooms', 'bedrooms', 'beds', 'price', 'guests_included', 'number_of_reviews',
                     'review_scores_rating','city']]

In [88]:
#add amnities_matrix which cotains data about presence and absence of amenities for each listing to features dataframe
features=pd.concat([features, pd.DataFrame(data=amenities_matrix.T, columns=amenities)], axis=1)
features

,host_listings_count,host_total_listings_count,accommodates,bathrooms,bedrooms,beds,price,guests_included,number_of_reviews,review_scores_rating,...,Suitable for Events,Suitable for events,TV,Washer,Washer / Dryer,Wheelchair Accessible,Wheelchair accessible,Wireless Internet,translation missing: en.hosting_amenity_49,translation missing: en.hosting_amenity_50
0,1.0,1.0,4,1.5,2.0,3.0,250,1,0,NaN,...,False,False,True,True,False,False,False,True,False,False
1,1.0,1.0,2,1.0,1.0,1.0,65,0,36,94.0,...,False,False,True,True,False,False,False,True,False,False
2,1.0,1.0,2,1.0,1.0,1.0,65,1,41,98.0,...,False,False,True,True,False,False,False,True,True,True
3,1.0,1.0,4,1.0,1.0,2.0,75,2,1,100.0,...,False,False,True,True,False,False,False,True,False,False
4,1.0,1.0,2,1.5,1.0,2.0,79,1,29,99.0,...,False,False,False,False,False,False,False,True,False,False
5,2.0,2.0,2,1.0,1.0,1.0,75,1,8,100.0,...,False,False,True,False,False,False,False,True,False,False
6,5.0,5.0,3,1.0,1.0,2.0,100,1,57,90.0,...,False,False,True,False,False,False,False,True,False,False
7,2.0,2.0,2,2.0,1.0,1.0,75,1,67,96.0,...,False,False,True,True,False,False,False,True,False,False
8,1.0,1.0,2,1.0,1.0,2.0,58,2,65,96.0,...,False,False,False,False,False,False,False,True,False,False
9,2.0,2.0,5,1.0,2.0,2.0,229,4,33,94.0,...,False,False,True,True,False,False,False,True,False,False


In [89]:
#some of the columns in features dataframe are saved as strings of the form "t" or "f". Map them to boolean values

for boolean_feature in ['host_is_superhost', 'host_identity_verified', 'host_has_profile_pic',
                   'is_location_exact', 'requires_license', 'instant_bookable',
                   'require_guest_profile_picture', 'require_guest_phone_verification']:
    features[boolean_feature] = listing[boolean_feature].map(lambda s: False if s == "f" else True)

In [90]:
#Encode city column to dummy variables using get_dummies function and add it to feautures dataframe

features = pd.concat([features, pd.get_dummies(listing['city']).astype(np.bool)], axis=1)
features

,host_listings_count,host_total_listings_count,accommodates,bathrooms,bedrooms,beds,price,guests_included,number_of_reviews,review_scores_rating,...,is_location_exact,requires_license,instant_bookable,require_guest_profile_picture,require_guest_phone_verification,boston,los-angeles,new-york-city,portland,san-francisco
0,1.0,1.0,4,1.5,2.0,3.0,250,1,0,NaN,...,True,False,False,False,False,True,False,False,False,False
1,1.0,1.0,2,1.0,1.0,1.0,65,0,36,94.0,...,True,False,True,False,False,True,False,False,False,False
2,1.0,1.0,2,1.0,1.0,1.0,65,1,41,98.0,...,True,False,False,True,False,True,False,False,False,False
3,1.0,1.0,4,1.0,1.0,2.0,75,2,1,100.0,...,False,False,False,False,False,True,False,False,False,False
4,1.0,1.0,2,1.5,1.0,2.0,79,1,29,99.0,...,True,False,False,False,False,True,False,False,False,False
5,2.0,2.0,2,1.0,1.0,1.0,75,1,8,100.0,...,True,False,False,False,False,True,False,False,False,False
6,5.0,5.0,3,1.0,1.0,2.0,100,1,57,90.0,...,False,False,False,False,False,True,False,False,False,False
7,2.0,2.0,2,2.0,1.0,1.0,75,1,67,96.0,...,True,False,False,True,True,True,False,False,False,False
8,1.0,1.0,2,1.0,1.0,2.0,58,2,65,96.0,...,True,False,False,False,False,True,False,False,False,False
9,2.0,2.0,5,1.0,2.0,2.0,229,4,33,94.0,...,True,False,False,False,False,True,False,False,False,False


In [91]:
#fill null values with the median of the columns

for col in features.columns[features.isnull().any()]:
    features[col] = features[col].fillna(features[col].median())

In [93]:
#writing to csv
features.to_csv('finaldata/listings_all_clean.csv')

In [3]:
#Creating sample file to pull to github
import pandas as pd
listing_sample=pd.read_csv("listings_all.csv", encoding = "ISO-8859-1",low_memory=False)
listing_sample.ix[:10000].to_csv("listing_all_sample.csv")


listing_sample=pd.read_csv("reviews_all.csv", encoding = "ISO-8859-1",low_memory=False)
listing_sample.ix[:10000].to_csv("reviews_all_sample.csv")


listing_sample=pd.read_csv("calendar_all.csv", encoding = "ISO-8859-1",low_memory=False)
listing_sample.ix[:10000].to_csv("calendar_all_sample.csv")
